# Logistic Regression, Linear Classification and Stochastic Gradient Descent


In [54]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file
#load dataset
d=load_svmlight_file('a9a')
d[1][np.where(d[1]==-1)]=0

In [98]:
x_train=d[0].todense()
y_train=d[1].reshape(len(d[1]),-1)
#add x0
m,n=np.shape(x_train)
x_train=np.column_stack((np.ones(m),x_train))

#add validset
d=load_svmlight_file('a9a.t')
d[1][np.where(d[1]==-1)]=0
x_valid=d[0].todense()
y_valid=d[1].reshape(len(d[1]),-1)
p,q=np.shape(x_valid)
x_valid=np.column_stack((np.ones(p),x_valid))


## NAG

In [88]:
gamma=0.9
eta=0.1
epochs=400
epochstep=100
epochset=[]
l_NAG=[]
grad=np.zeros((n+1,1))
v=np.zeros((n+1,1))
w=np.random.rand(n+1,1)
batch_size=128

def label(x):
    if x<0.5:
        return 0
    else:
        return 1
    
for i in range (epochs+1):
    #training
    batch_bgn=np.random.randint(0,m-batch_size-1)
    for j in range(batch_bgn,batch_bgn+batch_size):
        h=1/(1+np.exp(x_train[j]*(gamma*v-w)))
        grad=grad+((h-y_train[j])*x_train[j]).T
    grad=grad/batch_size
    v=gamma*v+eta*grad
    w=w-v
    if(i<100 and i%10==0) or (i%epochstep==0):
        epochset.append(i)
        #compute loss
        loss=0.
        correct=0
        for k in range(p):
            h=1/(1+np.exp(x_valid[k]*(gamma*v-w)))
            loss+=(y_valid[k]*np.log(h)+(1-y_valid[k])*np.log(1-h))
            if label(h)==y_valid[k]:
                correct+=1
        loss=np.asarray(-loss/p)
        l_NAG.append(loss[0][0])
        #print 
        print('Epoch',i)
        print('validation loss',loss[0][0])
        print('validation accuracy',correct/p)
        print('')


Epoch 0
validation loss 4.05750348184
validation accuracy 0.23622627602727106

Epoch 10
validation loss 1.23745542828
validation accuracy 0.7640808304158221

Epoch 20
validation loss 0.479039183219
validation accuracy 0.7660463116516184

Epoch 30
validation loss 0.402039975205
validation accuracy 0.8164731896075179

Epoch 40
validation loss 0.385760383116
validation accuracy 0.8253178551686015

Epoch 50
validation loss 0.369194612254
validation accuracy 0.82943308150605

Epoch 60
validation loss 0.358729046151
validation accuracy 0.8344696271727781

Epoch 70
validation loss 0.350639707007
validation accuracy 0.8380320619126589

Epoch 80
validation loss 0.342518266202
validation accuracy 0.8391376451077943

Epoch 90
validation loss 0.34269262662
validation accuracy 0.8404274921687857

Epoch 100
validation loss 0.338499904135
validation accuracy 0.8428843437135312

Epoch 200
validation loss 0.331641769149
validation accuracy 0.8459554081444629

Epoch 300
validation loss 0.330611760713
va

## RMSProp

In [90]:
gamma=0.9
eta=0.01
eps=1e-8
epochset=[]
l_RMSP=[]
G=np.zeros((n+1,1))
grad=np.zeros((n+1,1))
w=np.random.rand(n+1,1)

for i in range(epochs+1):
    #training
    batch_bgn=np.random.randint(m-batch_size-1)
    for j in range(batch_bgn,batch_bgn+batch_size):
        h=1/(1+np.exp(-x_train[j]*w))
        grad+=((h-y_train[j])*x_train[j]).T
    grad=grad/batch_size
    G=gamma*G+np.multiply((1-gamma)*grad,grad)
    w=w-np.multiply(eta/(np.sqrt(G+eps)),grad)
    if(i<100 and i%10==0) or (i%epochstep==0):
        epochset.append(i)
        #compute loss
        loss=0.
        correct=0
        for k in range(p):
            h=1/(1+np.exp(-x_valid[k]*w))
            loss+=(y_valid[k]*np.log(h)+(1-y_valid[k])*np.log(1-h))
            if label(h)==y_valid[k]:
                correct+=1
        loss=np.asarray(-loss/p)
        l_RMSP.append(loss[0][0])
        #print 
        print('Epoch',i)
        print('validation loss',loss[0][0])
        print('validation accuracy',correct/p)
        print('')



Epoch 0
validation loss 5.84413572011
validation accuracy 0.23622627602727106

Epoch 10
validation loss 4.15999776952
validation accuracy 0.23622627602727106

Epoch 20
validation loss 2.9464410294
validation accuracy 0.23622627602727106

Epoch 30
validation loss 1.94274596217
validation accuracy 0.23659480375898287

Epoch 40
validation loss 1.19089519936
validation accuracy 0.2781155948651803

Epoch 50
validation loss 0.757372692594
validation accuracy 0.5117621767704686

Epoch 60
validation loss 0.592837257165
validation accuracy 0.6919722375775443

Epoch 70
validation loss 0.507714458833
validation accuracy 0.7680117928874147

Epoch 80
validation loss 0.456741322162
validation accuracy 0.7942386831275721

Epoch 90
validation loss 0.424222303201
validation accuracy 0.807198575026104

Epoch 100
validation loss 0.39602110425
validation accuracy 0.8175787728026535

Epoch 200
validation loss 0.32979515863
validation accuracy 0.8484736809778269

Epoch 300
validation loss 0.326113599504
val

# AdaDelta

In [91]:
gamma=0.999
eps=1e-8
delta_t=np.zeros((n+1,1))
epochset=[]
l_Ada=[]
G=np.zeros((n+1,1))
grad=np.zeros((n+1,1))
w=np.random.rand(n+1,1)

for i in range(epochs+1):
    #training
    batch_bgn=np.random.randint(0,m-batch_size-1)
    for j in range(batch_bgn,batch_bgn+batch_size):
        h=1/(1+np.exp(-x_train[j]*w))
        grad+=((h-y_train[j])*x_train[j]).T
    grad=grad/batch_size
    G=gamma*G+np.multiply((1-gamma)*grad,grad)
    delta_w=-np.multiply(np.sqrt(delta_t+eps),grad)/np.sqrt(G+eps)
    delta_t=gamma*delta_t+np.multiply((1-gamma)*delta_w,delta_w)
    w=w+delta_w
    if(i<100 and i%10==0) or (i%epochstep==0):
        epochset.append(i)
        #compute loss
        loss=0.
        correct=0
        for k in range(p):
            h=1/(1+np.exp(-x_valid[k]*w))
            loss+=(y_valid[k]*np.log(h)+(1-y_valid[k])*np.log(1-h))
            if label(h)==y_valid[k]:
                correct+=1
        loss=np.asarray(-loss/p)
        l_Ada.append(loss[0][0])
        #print 
        print('Epoch',i)
        print('validation loss',loss[0][0])
        print('validation accuracy',correct/p)
        print('')

Epoch 0
validation loss 4.24743613913
validation accuracy 0.23622627602727106

Epoch 10
validation loss 3.89158476937
validation accuracy 0.23622627602727106

Epoch 20
validation loss 3.53721045077
validation accuracy 0.23622627602727106

Epoch 30
validation loss 3.18739045721
validation accuracy 0.23622627602727106

Epoch 40
validation loss 2.83853884736
validation accuracy 0.23622627602727106

Epoch 50
validation loss 2.50188581
validation accuracy 0.23622627602727106

Epoch 60
validation loss 2.19294222381
validation accuracy 0.23659480375898287

Epoch 70
validation loss 1.9007002211
validation accuracy 0.2387445488606351

Epoch 80
validation loss 1.64130059811
validation accuracy 0.24525520545421042

Epoch 90
validation loss 1.42068760238
validation accuracy 0.26171611080400464

Epoch 100
validation loss 1.23096860864
validation accuracy 0.2873287881579756

Epoch 200
validation loss 0.573446482896
validation accuracy 0.7116884712241263

Epoch 300
validation loss 0.484656401659
vali

## Adam

In [114]:
beta=0.88
gamma=0.99
eta=0.005
eps=1e-8
moment=np.random.rand(n+1,1)
epochset=[]
l_Adam=[]
G=np.zeros((n+1,1))
grad=np.zeros((n+1,1))
w=np.random.rand(n+1,1)

for i in range(epochs+1):
    #training
    batch_bgn=np.random.randint(0,m-batch_size-1)

    for j in range(batch_bgn,batch_bgn+batch_size):
        h=1/(1+np.exp(-x_train[j]*w))
        grad+=((h-y_train[j])*x_train[j]).T
    grad=grad/batch_size
    moment=beta*moment+(1-beta)*grad
    G=gamma*G+np.multiply((1-gamma)*grad,grad)
    alpha=eta*np.sqrt(1-gamma)/(1-beta)
    #if(i>0):
     #   beta=beta/np.sqrt(i)
      #  eta=eta/np.sqrt(i)
    w=w-alpha*moment/np.sqrt(G+eps)
    if(i<100 and i%10==0) or (i%epochstep==0):
        epochset.append(i)
        #compute loss
        loss=0.
        correct=0
        for k in range(p):
            h=1/(1+np.exp(-x_valid[k]*w))
            loss+=(y_valid[k]*np.log(h)+(1-y_valid[k])*np.log(1-h))
            if label(h)==y_valid[k]:
                correct+=1
        loss=np.asarray(-loss/p)
        l_Adam.append(loss[0][0])
        #print 
        print('Epoch',i)
        print('validation loss',loss[0][0])
        print('validation accuracy',correct/p)
        print('')

Epoch 0
validation loss 3.87561910779
validation accuracy 0.28001965481235797

Epoch 10
validation loss 3.64961446912
validation accuracy 0.5296971930471102

Epoch 20
validation loss 3.87584931228
validation accuracy 0.6319636385971378

Epoch 30
validation loss 3.86188786405
validation accuracy 0.6630428106381672

Epoch 40
validation loss 3.78318926812
validation accuracy 0.6626128616178367

Epoch 50
validation loss 3.7088770273
validation accuracy 0.6615072784227013

Epoch 60
validation loss 3.6391502746
validation accuracy 0.6546280940974142

Epoch 70
validation loss 3.58122796713
validation accuracy 0.6547509366746515

Epoch 80
validation loss 3.5313533847
validation accuracy 0.6653153983170567

Epoch 90
validation loss 3.48490096044
validation accuracy 0.6756955960936061

Epoch 100
validation loss 3.44279273231
validation accuracy 0.6863214790246299

Epoch 200
validation loss 3.14481095571
validation accuracy 0.728579325594251

Epoch 300
validation loss 2.96691773781
validation acc

In [120]:
#draw plot
def draw_plot(epochset,l_NAG,l_RMSP,l_Ada,l_Adam):
    plt.figure(figsize=(10,15))
    plt.subplot(211)
    plt.xlabel('epochs')
    plt.ylabel('loss')
    plt.plot(epochset,l_NAG,color='red',label='NAG',linewidth=1.5,linestyle='-')
    plt.plot(epochset,l_RMSP,color='yellow',label='RMSProp',linewidth=1.5,linestyle='-')
    plt.plot(epochset,l_Ada,color='blue',label='AdaDelta',linewidth=1.5,linestyle='-')
    plt.plot(epochset,l_Adam,color='green',label='Adam',linewidth=1.5,linestyle='-')
    plt.legend(loc='upper right')

draw_plot(epochset,l_NAG,l_RMSP,l_Ada,l_Adam)
